It is widely regarded taht Taylor Swift's 'eras' have a disticnt sound signature and a distint writting style, the (typically) two years between every album realease are spent in her 'reinventing' her style and she produces something diffferent every era, so how true is that?

If that is a 100% true a clustering algorithm should place all albums (evermore and folklore are in one _technically_) in seperate clusters, but that is obvoiusly not going to happen, so lets investigate

In [38]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import random
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

In [39]:
cid ='d888370f9c2a4514aade84a1b6ae976e'
secret ='01774fdc665e4dcc8dc8c48667e46bf1'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [44]:
l = ['7eX5SypK35V8Y9d9pS6rWy',
        '4fPBB44eDH71YohayI4eKV',
        #'2M7UdnD0fEaryh8TnCvqFX',
        #'6LqZFfv4fUP7va14Y6VW9a',
]

print(sp.audio_features(tracks=l))

[{'danceability': 0.472, 'energy': 0.746, 'key': 11, 'loudness': -4.816, 'mode': 1, 'speechiness': 0.0321, 'acousticness': 0.00581, 'instrumentalness': 0.663, 'liveness': 0.0999, 'valence': 0.403, 'tempo': 97.44, 'type': 'audio_features', 'id': '7eX5SypK35V8Y9d9pS6rWy', 'uri': 'spotify:track:7eX5SypK35V8Y9d9pS6rWy', 'track_href': 'https://api.spotify.com/v1/tracks/7eX5SypK35V8Y9d9pS6rWy', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7eX5SypK35V8Y9d9pS6rWy', 'duration_ms': 162040, 'time_signature': 4}, {'danceability': 0.63, 'energy': 0.908, 'key': 11, 'loudness': -2.42, 'mode': 1, 'speechiness': 0.0326, 'acousticness': 0.0238, 'instrumentalness': 0.592, 'liveness': 0.116, 'valence': 0.968, 'tempo': 120.522, 'type': 'audio_features', 'id': '4fPBB44eDH71YohayI4eKV', 'uri': 'spotify:track:4fPBB44eDH71YohayI4eKV', 'track_href': 'https://api.spotify.com/v1/tracks/4fPBB44eDH71YohayI4eKV', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4fPBB44eDH71YohayI4eKV', 'durat

In [37]:
# df = pd.read_csv('spotify_hist_sept.csv')

# df.query('uri==3kKb3U3a8jmUJQB9ypE1Jl')['track name']
# print(df.loc[df['uri'] == '3kKb3U3a8jmUJQB9ypE1Jl', 'track_name'].iloc[0])
# df.head(10)


HAIM


In [45]:
def get_track_features(df):
    uri_real = []
    danceability = []
    energy = []
    key = []
    loudness = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    duration_ms = []

    for i in df['uri']:
        print(df.loc[df['uri'] == i, 'track_name'].iloc[0])
        for x in sp.audio_features(tracks=[i]):
            # uri_real.append(x['uri'])
            danceability.append(x['danceability'])
            energy.append(x['energy'])
            key.append(x['key'])
            loudness.append(x['loudness'])
            speechiness.append(x['speechiness'])
            acousticness.append(x['acousticness'])
            instrumentalness.append(x['instrumentalness'])
            liveness.append(x['liveness'])
            valence.append(x['valence'])
            tempo.append(x['tempo'])
            duration_ms.append(x['duration_ms'])
            
    df2 = pd.DataFrame({
    # 'uri_real': uri_real,
    'danceability':danceability,
    'energy':energy,
    'key':key,
    'loudness':loudness,
    'speechiness':speechiness,
    'acousticness':acousticness,
    'instrumentalness':instrumentalness,
    'liveness':liveness,
    'valence':valence,
    'tempo':tempo,
    'duration_ms':duration_ms})

    df3 = df.merge(df2, left_index= True, right_index= True)
    
    return df3

In [46]:
def get_album_tracks_and_features(uri_info):
    uri = []
    track = []
    duration = []
    explicit = []
    track_number = []
    album_artist = sp.album(uri_info)
    album = album_artist['name']
    artist = album_artist['artists'][0]['name']
    one = sp.album_tracks(uri_info, limit=50, offset=0, market='US')
    df1 = pd.DataFrame(one)
    
    for i, x in df1['items'].items():
        uri.append(x['uri'])
        track.append(x['name'])
        explicit.append(x['explicit'])
        track_number.append(x['track_number'])
    
    # print(uri)
    # print(track)
    print(album)
    # print(explicit)
    # print(track_number)
    df2 = pd.DataFrame({
    'uri':uri,
    'track_number':track_number,
    'track_name':track,
    'album':album,
    'artist':artist,
    'explicit':explicit})

    df3 = get_track_features(df2)
    
    return df3

In [48]:
with open('album_links.txt', 'r') as f:
    albums = [l.rstrip() for l in f.readlines()]

frames = []
for i in albums:
    df_interim = get_album_tracks_and_features(i)
    frames.append(df_interim)

df = pd.concat(frames, ignore_index=True)

df.to_csv('taylor_discography_features.csv', index=False)
df.head()

evermore (deluxe version)
willow
champagne problems
gold rush
‘tis the damn season
tolerate it
no body, no crime (feat. HAIM)
happiness
dorothea
coney island (feat. The National)
ivy
cowboy like me
long story short
marjorie
closure
evermore (feat. Bon Iver)
right where you left me - bonus track
it’s time to go - bonus track
Fearless (Taylor's Version)
Fearless (Taylor’s Version)
Fifteen (Taylor’s Version)
Love Story (Taylor’s Version)
Hey Stephen (Taylor’s Version)
White Horse (Taylor’s Version)
You Belong With Me (Taylor’s Version)
Breathe (feat. Colbie Caillat) (Taylor’s Version)
Tell Me Why (Taylor’s Version)
You’re Not Sorry (Taylor’s Version)
The Way I Loved You (Taylor’s Version)
Forever & Always (Taylor’s Version)
The Best Day (Taylor’s Version)
Change (Taylor’s Version)
Jump Then Fall (Taylor’s Version)
Untouchable (Taylor’s Version)
Forever & Always (Piano Version) (Taylor’s Version)
Come In With The Rain (Taylor’s Version)
Superstar (Taylor’s Version)
The Other Side Of The Do

,uri,track_number,track_name,album,artist,explicit,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,spotify:track:2gVhfX2Gy1T9kDuS9azrF7,1,willow,evermore (deluxe version),Taylor Swift,False,0.392,0.574,7,-9.195,0.1700,0.833,0.001790,0.1450,0.529,81.112,214707
1,spotify:track:1gcyHQpBQ1lfXGdhZmWrHP,2,champagne problems,evermore (deluxe version),Taylor Swift,True,0.462,0.240,0,-12.077,0.0377,0.920,0.000000,0.1130,0.320,171.319,244000
2,spotify:track:3Dby3p1m6IOZn2gIIqECgK,3,gold rush,evermore (deluxe version),Taylor Swift,True,0.512,0.462,9,-10.491,0.0408,0.830,0.166000,0.1210,0.353,112.050,185320
3,spotify:track:6sQckd3Z8NPxVVKUnavY1F,4,‘tis the damn season,evermore (deluxe version),Taylor Swift,False,0.575,0.434,5,-8.193,0.0312,0.735,0.000066,0.1050,0.348,145.916,229840
4,spotify:track:6lCvK2AR2uOKkVFCVlAzzm,5,tolerate it,evermore (deluxe version),Taylor Swift,True,0.316,0.361,9,-10.381,0.0488,0.878,0.000027,0.0797,0.221,74.952,245440


In [113]:
df = pd.read_csv('taylor_discography_features.csv')

df.head()

,uri,track_number,track_name,album,artist,explicit,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,spotify:track:2gVhfX2Gy1T9kDuS9azrF7,1,willow,evermore (deluxe version),Taylor Swift,False,0.392,0.574,7,-9.195,0.1700,0.833,0.001790,0.1450,0.529,81.112,214707
1,spotify:track:1gcyHQpBQ1lfXGdhZmWrHP,2,champagne problems,evermore (deluxe version),Taylor Swift,True,0.462,0.240,0,-12.077,0.0377,0.920,0.000000,0.1130,0.320,171.319,244000
2,spotify:track:3Dby3p1m6IOZn2gIIqECgK,3,gold rush,evermore (deluxe version),Taylor Swift,True,0.512,0.462,9,-10.491,0.0408,0.830,0.166000,0.1210,0.353,112.050,185320
3,spotify:track:6sQckd3Z8NPxVVKUnavY1F,4,‘tis the damn season,evermore (deluxe version),Taylor Swift,False,0.575,0.434,5,-8.193,0.0312,0.735,0.000066,0.1050,0.348,145.916,229840
4,spotify:track:6lCvK2AR2uOKkVFCVlAzzm,5,tolerate it,evermore (deluxe version),Taylor Swift,True,0.316,0.361,9,-10.381,0.0488,0.878,0.000027,0.0797,0.221,74.952,245440


def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

df_features = parallelize_dataframe(df, get_track_features)

In [116]:
from requests import exceptions
import lyricsgenius as lg

token = '0GGXQLMawu2kxxYNqTGFT79I0wq8McYb8jbOz0HGOQc-sCAmu0HuYKo_C71dsRg-'
genius = lg.Genius(token)

def get_lyrics(song_name, artist_name):

    if '(' in song_name and 'Taylor\'s Version' not in song_name:
        song_name = song_name.split('(')[0]


    songs = genius.search_songs(song_name + ' ' + artist_name)

    # url = songs['hits']
    url = songs['hits'][0]['result']['url']
    print(url)
    try:
        song_lyrics = genius.lyrics(song_url=url)
    except requests.exceptions.Timeout:
        try:
            song_lyrics = genius.lyrics(song_url=url)
        except requests.exceptions.Timeout:
            song_lyrics = 'timeout'
    
    if song_lyrics is None:
        song_lyrics = 'none for some reason'

    return song_lyrics.split('\n')


    # for song in songs['hits']:
    #     url = song['result']['url']
    #     song_lyrics = genius.lyrics(song_url=url)
    #     # id = song['result']['id']
    #     # song_lyrics = genius.lyrics(id)
    #     lyrics.append(song_lyrics)
# print(get_lyrics('We Were Happy (Taylor’s Version) (From The Vault)', 'Taylor Swift'))



def lyrics_onto_frame(df1):
    for i, v in enumerate(zip(df1['track_name'], df1['artist'])):
        test = get_lyrics(v[0], v[1])
        print(i)
        df1.loc[df.index[i], 'lyrics'] = test

df['lyrics'] = df.apply(lambda lyric: get_lyrics(lyric.track_name, lyric.artist), axis=1)





https://genius.com/Taylor-swift-willow-lyrics
https://genius.com/Taylor-swift-champagne-problems-lyrics
https://genius.com/Taylor-swift-gold-rush-lyrics
https://genius.com/Taylor-swift-tis-the-damn-season-lyrics
https://genius.com/Taylor-swift-tolerate-it-lyrics
https://genius.com/Taylor-swift-no-body-no-crime-lyrics
https://genius.com/Taylor-swift-happiness-lyrics
https://genius.com/Taylor-swift-dorothea-lyrics
https://genius.com/Taylor-swift-coney-island-lyrics
https://genius.com/Taylor-swift-ivy-lyrics
https://genius.com/Taylor-swift-cowboy-like-me-lyrics
https://genius.com/Milotic-milos-2022-listening-log-annotated
https://genius.com/Taylor-swift-marjorie-lyrics
https://genius.com/Taylor-swift-closure-lyrics
https://genius.com/Taylor-swift-evermore-lyrics
https://genius.com/Genius-users-2017-user-interview-an-enlightening-madkool-convo-with-madkol-annotated
https://genius.com/Spotify-new-music-friday-1-08-21-annotated
https://genius.com/Taylor-swift-fearless-lyrics
https://genius.c

In [118]:
df.to_csv('taylor_discography_features_and_lyrics.csv', index=False)


 - Explicit: The indicator of whether the lyric contains explicit words or expressions.
 - Danceability: The degree of how suitable a track is for dancing based on tempo, rhythm stability, beat strength, and overall regularity. (0~1)
 - Energy: The perceptual measure of intensity based on dynamic range, perceived loudness, timbre, onset rate, and general entropy. (0~1)
 - Key: The estimated overall pitch class of the track and its type of scale from which its melodic content is derived (0 - 11).
 - Loudness: The quality of a sound that is the primary psychological correlate of amplitude in decibel. (-60~0)
 - Speechiness: The presence of spoken words in a track. (0~1) maybe 0 to 0.5
 - Acousticness: The confidence measure whether the track is acoustic. (0~1)
 - Instrumentalness: The confidence measure taht the track is instrumental? (0 - 1)
 - Liveness: The presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. (0~1) _can be dropped_
 - Valence: The musical positiveness conveyed by a track (e.g. happy, cheerful, euphoric). (0~1)
 - Tempo: The overall estimated tempo of a track in beats per minute (BPM). (50~200)
 - Duration: The length of the track in seconds. wont use in the radar plot

In [122]:
df.loc[5, 'lyrics']

['\u200bno body, no crime Lyrics[Intro: HAIM]',
 'He did it',
 'He did it',
 '',
 '[Verse 1: Taylor Swift]',
 "Este's a friend of mine",
 'We meet up every Tuesday night for dinner and a glass of wine',
 "Este's been losin' sleep",
 "Her husband's actin' different, and it smells like infidelity",
 'She says, "That ain\'t my Merlot on his mouth',
 'That ain\'t my jewelry on our joint account"',
 "No, there ain't no doubt",
 "I think I'm gonna call him out",
 '[Chorus: Taylor Swift & HAIM]',
 'She says, "I think he did it, but I just can\'t prove it"',
 "I think he did it, but I just can't prove it",
 "I think he did it, but I just can't prove it",
 'No, no body, no crime',
 "But I ain't lettin' up until the day I die",
 '',
 '[Post-Chorus: HAIM]',
 'No, no',
 'I think he did it',
 'No, no',
 'He did it',
 '',
 '[Verse 2: Taylor Swift]',
 "Este wasn't there Tuesday night at Olive Gardеn",
 'At her job or anywhere',
 'Hе reports his missing wife',
 'And I noticed when I passed his house',

In [4]:


df_features['duration_m_s_f'] = pd.to_datetime(df_features['duration_ms'], unit='ms')
df_features['duration_m_s_f'] = df_features['duration_m_s_f'].dt.strftime('%M:%S.%f')

df_features['tempo'] = df_features['tempo'].div(210)
df_features['loudness'] = df_features['loudness']/(-60)
df_features['key'] = df_features['key']/11
df_features['speechiness'] = df_features['speechiness'] / 0.7 

theta = list(df_features.columns)[6:16]
r = list(df_features.loc[145])[6:16]

d = dict(zip(theta, r))

# df_features.describe()

# df_plot = df_features.copy(deep=True)
# df_plot.drop(['date', 'uri', 'link', 'duration_ms'], axis=1, inplace=True)
# df_plot.head(10)

# df_plot['tempo'] = df_plot['tempo'].div(210)
# # df_plot['duration_ms'] = df_plot['duration_ms'].div(df_plot['duration_ms'].max())
# df_plot['loudness'] = df_plot['loudness']/df_plot['loudness'].min()
# df_plot['key'] = df_plot['key']/df_plot['key'].max()
# # df_plot['liveness'] = df_plot['liveness'] * 2  # <--- may need to entirely drop this col
# df_plot['speechiness'] = df_plot['speechiness'] / 0.7  # <--- may need to entirely drop this col


# df_avgs = df_plot.describe()
# df_avgs.head(15)


In [20]:
[i for i in r]
r

[0.548,
 0.611,
 0.09090909090909091,
 0.15,
 0.18714285714285717,
 0.17,
 0.00247,
 0.0762,
 0.725,
 0.9618333333333334]

In [25]:
# import plotly.graph_objects as go
# import plotly.offline as pyo
# import plotly.express as px
# import nbformat

import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

colours = ['aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure',
        'beige', 'bisque', 'black', 'blanchedalmond', 'blue',
        'blueviolet', 'brown', 'burlywood', 'cadetblue',
        'chartreuse', 'chocolate', 'coral', 'cornflowerblue',
        'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan',
        'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen',
        'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange',
        'darkorchid', 'darkred', 'darksalmon', 'darkseagreen',
        'darkslateblue', 'darkslategray', 'darkslategrey',
        'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue',
        'dimgray', 'dimgrey', 'dodgerblue', 'firebrick',]

song_number = 123
song_list = list(range(0, 10))
# song_list = [i + 122 for i in song_list]


fig = go.Figure()


for i, v in enumerate(song_list):

    r = list(df_features.loc[v])[6:16]
    fig.add_trace(go.Scatterpolar(
            r = r,
            theta = theta,
            mode = 'markers',
            name = df_features['track_name'].loc[v],
            #line_color = colours[-1*i]
        ),)

fig.update_layout(
    template='ggplot2',
    polar = dict(
        radialaxis = dict(range=[0, 1]),
    )
)

fig.update_traces(fill='toself')
fig.update_layout(showlegend=True)
fig.show(renderer="browser")

# theta2 = list(df_avgs.columns)
# print(theta2)
# mean = df_avgs.iloc[[1]].values[0]
# print(mean)

# dfnew = pd.DataFrame(dict(r = mean, theta = theta2))

# print(dfnew.head(10))

# fig = px.line_polar(dfnew, r='r', theta='theta', line_close=True)
# fig.update_traces(fill='toself')
# fig.show(renderer="browser")



NameError: name 'df_features' is not defined

In [51]:
import sweetviz as sv

analyze_report = sv.analyze(df_features)

c:\Users\Ayush\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


In [100]:
analyze_report.show_html()

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


maybe the metrics to really focus on are:

 - Danceability
 - Energy
 - Key
 - loudness
 - speechiness
 - acousticness
 - 